Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [ ]:
from itertools import combinations
from tqdm import tqdm
import numpy as np
import csv
import time

## Simple Test Problem

In [141]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load('lab2/test_problem.npy')

## Common tests

In [142]:
problem = np.load('lab2/test_problem.npy')

In [143]:
# Negative values?
np.any(problem < 0)

np.False_

In [144]:
# Diagonal is all zero?
np.allclose(np.diag(problem), 0.0)

True

In [145]:
# Symmetric matrix?
np.allclose(problem, problem.T)

True

In [146]:
# Triangular inequality
all(
    problem[x, y] <= problem[x, z] + problem[z, y]
    for x, y, z in list(combinations(range(problem.shape[0]), 3))
)

True

### TSP - solution

In [ ]:
# Genetic Algorithm with turnament selection, order-based crossover and swap mutation
def ga(problem, population_size=100, generations=1000, turnament_size=5, swap_mutation=True):
    population = [
        np.random.permutation(problem.shape[0]) for _ in range(population_size)
    ]

    def fitness(individual):
        return sum(
            problem[individual[i], individual[(i + 1) % problem.shape[0]]]
            for i in range(problem.shape[0])
        )
    
    def turnament_selection(population):
        selected = np.random.choice(population_size, turnament_size, replace=False)
        best = min(selected, key=lambda i: fitness(population[i]))
        return population[best]

    def ob_crossover(p1, p2):
        size = len(p1)
        child = [-1] * size
        start, end = sorted(np.random.choice(range(size), 2, replace=False))
        child[start:end] = p1[start:end]
        p2_indices = [i for i in range(size) if p2[i] not in child]
        c_index = end % size
        for i in p2_indices:
            if child[c_index] == -1:
                child[c_index] = p2[i]
                c_index = (c_index + 1) % size
        return np.array(child)
    
    mutation_rate = 1/problem.shape[0]

    def mutate(individual):
        for i in range(problem.shape[0]):
            if np.random.rand() < mutation_rate:
                j = np.random.randint(0, problem.shape[0])
                individual[i], individual[j] = individual[j], individual[i]
        return individual
    
    for _ in tqdm(range(generations)):
        population = sorted(population, key=fitness)
        offspring = population[:population_size//10]  # 10% individual elitism
        while len(offspring) < population_size:
            parents = turnament_selection(population), turnament_selection(population) # there is a chance to select the same parent twice
            child = ob_crossover(parents[0], parents[1])
            if swap_mutation:
                child = mutate(child)
            offspring.append(child)
        population = offspring

    best_individual = min(population, key=fitness)
    return best_individual, fitness(best_individual)

In [ ]:
# remove higher size problems for speedup execution
problems = ["problem_g_10.npy", "problem_g_20.npy", "problem_g_50.npy", "problem_g_100.npy", "problem_g_200.npy", "problem_g_500.npy", "problem_g_1000.npy",
            "problem_r1_10.npy", "problem_r1_20.npy", "problem_r1_50.npy", "problem_r1_100.npy", "problem_r1_200.npy", "problem_r1_500.npy", "problem_r1_1000.npy",
             "problem_r2_10.npy", "problem_r2_20.npy", "problem_r2_50.npy", "problem_r2_100.npy", "problem_r2_200.npy", "problem_r2_500.npy", "problem_r2_1000.npy"
           ]
for p in problems:
    prob = np.load(f'lab2/{p}')
    solution, cost = ga(prob)
    print(f"Problem: {p}\nBest route: {solution}\nCost: {cost}")

100%|██████████| 1000/1000 [00:07<00:00, 140.10it/s]


Problem: problem_g_10.npy
Best route: [2 8 0 7 9 5 4 6 1 3]
Cost: 1497.6636482252907


100%|██████████| 1000/1000 [00:22<00:00, 45.24it/s]


Problem: problem_g_50.npy
Best route: [31 20 18 46 47 14  0 27 30  5 25 40 48 28 15 13 35 16 23 45 44  6  7 34
 10  3 11 22 43  2  4 29 32 42  9 19 24 39 17 26  8 38 49 12  1 41 36 33
 21 37]
Cost: 2936.464811389757


100%|██████████| 1000/1000 [00:42<00:00, 23.56it/s]


...

Problem: problem_r2_200.npy
Best route: [ 39 147 130 199  69  83  90  62 100  52  27 175  70   6 152 143 128  34
 103  37 173 102 193  45 114 182  28  73   0 170 197  88 150  81 196 187
 171  65 176  95  21 144 119  55 166  98 146  51  49   2 190  36  48 148
 120 134 106 107 111  78 126  56 127 109 178   1  15 122 192  63  74  24
  25  10 155  68  67  14 167  77 140  93  84  54 158  92  71 184 141 133
 163  47   9  11  64  80  59  32 129 142  46 174  26  66  82   8  91  44
 168 169 115 180 198 189 112  87  57  94 138 157 105 149   7  40  76  53
 145 161 154 131  50 185 160 165 151  22 159 188  31  60 121 132  30  99
  18 118  89  41  96   3  79 186  85  13 179 194  33 108  43  42 156  20
  35 104 183 116  23 137  72 113  29 110 195 124 117  75 123 162  19  17
  38 139   4 135 136  58  12 181 177 191  61 153 172 125  97   5 164  16
  86 101]
Cost: -7841.709831057929


100%|██████████| 1000/1000 [07:58<00:00,  2.09it/s]


Problem: problem_r2_500.npy
Best route: [ 54 375 469 201  70 266 400 118   1  12  69 251   0 474 207 394 393 221
 234 350 197 255 183 422  97 273 373 212 466 278 228 406 249 160 417 308
 120 101 465  33 112 463 489 360  94  64 427 408  35  68 291 263  59 298
  98 371 222 395 459 206 493 379 364 109 125 190 140 442  72 392 473 366
 294 198 296 432 419 286 495  65 110 193  82 347 461 144  22 312   8 243
  73 333 486 220 362 358 117 423 338 387 441 245 304 289 410 462 437 254
 361 233 307 388 305 107 158 293 208 344 264 175  67 216  47 412  13 186
   2 302 181 163 242 376 345  15 230 172 131 268 339 431 166 180 494  10
 113  17 137   6 165 139   9  25 235 488 418 217 178 176 374 359 188 167
 381   4 299  51 295 124 111 104  48 192 342 368 223 389 276  23 297  75
 301 318  37 232  66 315 149 430 398 203 256  99 288 436 477 310  30 449
 162 450 231 353 205  16 326 196 390 108 401 282 241  36 262 236   7 153
 446 275  45 214 453 103 154 356  88 210 467 484  21 261 313  53 135  14
  34 483  2

100%|██████████| 1000/1000 [28:41<00:00,  1.72s/it]

Problem: problem_r2_1000.npy
Best route: [503 955 730  18 136 751 338 521 539 825 766 707 505 806 710 509 304 158
 907 980 662 432 353 912 398 997 130 483 834 337 999  99 669 573 545 104
 703 856 747 395 622 384 779 265 360  47 624 784 511 228  36 880 271  45
 238 929  20 441 413 393 150 242 231 131 515 892 215 406 862 992 420 154
 365 161 735 778 465 582 920 682 936 174 979 846 391 903 946 771 244 600
 698 380 861 944 333 824  27 863 440 899 444 200 651 375 820 118 729 706
 840 119 153 873 132 964  87 163  93 178 560 971 595 519 499 528 510 433
 537 974 475 850  35 965 203 149 114 232 345 459  48 578 599 287 939 983
 245  85 263 351 604  68 214 931  56 538 469 322 575 723 157 617 666 139
 489 854 205 401 988 740 486  77 500 565  63 725 282  29 700 307 523 402
 945 455 193 956 713 919 392 744  41 704  31 879 257 574 281 517 138 684
 954 508 183 981 173 285 170 882 186 893 317   5 471 733 896 107 885 588
 625 993 702  86 195 169 883 319 137 996 213 783 275 970 953 994 369 175
 764 240  

### compute stats on 10 runs

In [ ]:
PROBLEMS = ["problem_g_10.npy", "problem_g_20.npy", "problem_g_50.npy", "problem_g_100.npy", "problem_g_200.npy",
            "problem_r1_10.npy", "problem_r1_20.npy", "problem_r1_50.npy", "problem_r1_100.npy", "problem_r1_200.npy",
             "problem_r2_10.npy", "problem_r2_20.npy", "problem_r2_50.npy", "problem_r2_100.npy", "problem_r2_200.npy"
           ]

N_REPEATS = 10

results = []

print("Starting GA runs (population=100, generations=1000), 10 repeats per problem")
for p in PROBLEMS:
    print(f"\n== Running {p.replace('.npy','')} ==")
    prob = np.load(f'lab2/{p}')
    costs = []
    times = []
    for i in range(N_REPEATS):
        print(f" Run {i+1}/{N_REPEATS}...", end="", flush=True)
        start = time.perf_counter()
        _, cost = ga(prob)
        elapsed = time.perf_counter() - start
        costs.append(cost)
        times.append(elapsed)
        print(f" done (cost={cost:.4f}, time={elapsed:.2f}s)")

    best_cost = float(np.min(costs))
    avg_cost = float(np.mean(costs))
    avg_time = float(np.mean(times))
    results.append({
        "problem": p.replace('.npy','').replace('lab2/',''),
        "best_distance": best_cost,
        "average_distance": avg_cost,
        "runtime_s": avg_time,
        "runs": N_REPEATS
    })

# write CSV
csv_path = 'results/ga_results.csv'
with open(csv_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['problem','best_distance','average_distance','runtime_s','runs'])
    writer.writeheader()
    for r in results:
        writer.writerow(r)

# write markdown table
md_path = 'results/ga_results.md'
with open(md_path, 'w') as md:
    md.write('| Problem name | Best distance | Average distance | Runtime (s) | Notes |\n')
    md.write('|---|---:|---:|---:|\n')
    for r in results:
        md.write(f"| {r['problem']} | {r['best_distance']:.4f} | {r['average_distance']:.4f} | {r['runtime_s']:.2f} | |\n")